In [ ]:
%run ../utils/commonImports.py
%run ../utils/tradingImports.py
%matplotlib inline

# Load Data

# EMA

In [ ]:
def calculate_EMA(history, col, time_period):
    return calculateTalib('EMA', {col: history[col].values}, {'timeperiod': time_period})


def add_EMA(df):
    df['blue'] = calculate_EMA(df, 'close', 8)
    df['green'] = calculate_EMA(df, 'close', 13)
    df['yellow'] = calculate_EMA(df, 'close', 21)
    df['red'] = calculate_EMA(df, 'close', 55)
    return df

In [ ]:
def load_and_divide(file, from_date=None, to_date=None, train_ratio=0.7):
    data = load_trading_data(file, from_date, to_date)
    data = add_EMA(data)
    data_train, data_test = divide_train_and_test(data, train_ratio)
    return data, data_train, data_test


file = 'D:\\Dropbox\\My work\\krypl-project\\dataFeatures\\poloniex\\5min\\USDT_BTC_5min_2016-01-01_2017-12-31.tsv'
data, data_train, data_test = load_and_divide(file, from_date='2017-01-01')

file = 'D:\\Dropbox\\My work\\krypl-project\\dataCleaned\\poloniex\\5min\\USDT_BTC_5min_2016-01-01_2017-12-31.tsv'
ohlc, ohlc_train, ohlc_test = load_and_divide(file, from_date='2017-01-01')


features = ['red', 'yellow', 'green', 'blue']
data_manager_train = CurrencyDataManager(data_train['close'], data_train[features])
data_manager_test = CurrencyDataManager(data_test['close'], data_test[features])
wallet = {'usdt': 1000}
contract_pair = ContractPair.new('usdt', 'btc')

# Explore

In [ ]:
ohlcTrainCorrectPattern = ohlc_train.query("red < yellow < green < blue")

In [ ]:
ohlcTrainCorrectPattern.shape

# Implementation

In [ ]:
from modeling.strategy import Strategy


class CustomStrategy(Strategy):
    def __init__(self, exchange, dataManager, contractPair, tradeSize, willingLoss, targetProfit):
        super().__init__(exchange, dataManager, contractPair, tradeSize, willingLoss, targetProfit)
        self.history_len = 1
    
    @staticmethod
    def has_correct_order(red, yellow, green, blue):
        return red < yellow < green < blue
    
    @staticmethod
    def is_pattern_destroyed(red, yellow, green, blue):
        return (red > yellow) or (red > green) or (red > blue)
    
    def trade(self):
        
        in_row = 0
        after_sell = False
        while self.data_manager.has_tick():            
            history, price = self.data_manager.tick(self.history_len)
            
            if history.shape[0] == 0:
                continue
            
            red, yellow, green, blue = history[0]
            if np.isnan(red):
                continue

            if self.has_correct_order(red, yellow, green, blue):
                in_row += 1
            else:
                in_row = 0
                after_sell = False
            
            if not self.opened and not after_sell and in_row >= 1:
                self.buy(price)
                price_bought = price
            elif self.opened and self.is_risky(price_bought, price):
                self.sell_all(price)
                after_sell = True
            elif self.opened and self.is_target_satisfied(price_bought, price):
                self.sell_all(price)
                after_sell = True
            elif self.opened and self.is_pattern_destroyed(red, yellow, green, blue):
                self.sell_all(price)
                after_sell = True
        
        if self.opened:
            self.sell_all(price)


# Learning

In [ ]:
dataManager = deepcopy(data_manager_train)
exchange = BackTestExchange(dataManager, deepcopy(wallet), 0.0025)
strategy = CustomStrategy(exchange, dataManager, contract_pair, 100, willingLoss=0.5, targetProfit=0.05)
strategy.trade()
strategy.stats('usdt').report()

In [ ]:
exchange.wallet

In [ ]:
import time, sys
DAY = 288

allStats = None
print('historyLen willingLoss distanceFromMaxThreshold1 distanceFromMaxThreshold2 targetProfit')
for historyLen in [DAY, 2*DAY, 3*DAY, 7*DAY]:
    for willingLoss in [.01, 0.02, .03, .05, .1, .2, .5]:
        distanceThresholds = [.01, .02, .03, .04, .05, 0.06, .07, .08]
        for distanceFromMaxThreshold1 in distanceThresholds:
            for distanceFromMaxThreshold2 in [x for x in distanceThresholds if x <= distanceFromMaxThreshold1]:
                for targetProfit in [.01, .02, .03, .04, .05, .07, .1, .2, .3]:
                    s = "\r{} {} {} {} {}".format(historyLen, willingLoss, distanceFromMaxThreshold1, 
                                                  distanceFromMaxThreshold2, targetProfit)
                    sys.stdout.write(s)
                    sys.stdout.flush()

                    start_time = time.time()
                    dataManager = deepcopy(data_manager_train)
                    exchange = BackTestExchange(dataManager, deepcopy(wallet), 0.0025)
                    strategy = CustomStrategy(exchange, dataManager, historyLen, contract_pair, 100, willingLoss=willingLoss,
                                              distanceFromMaxThreshold1=distanceFromMaxThreshold1,
                                              distanceFromMaxThreshold2=distanceFromMaxThreshold2,
                                              targetProfit=targetProfit)
                    strategy.trade()
                    stats = strategy.stats('usdt').report().transpose().reset_index().rename(columns={'index': 'contract'})
                    stats['historyLen'] = historyLen
                    stats['willingLoss'] = willingLoss
                    stats['distanceFromMaxThreshold1'] = distanceFromMaxThreshold1
                    stats['distanceFromMaxThreshold2'] = distanceFromMaxThreshold2
                    stats['targetProfit'] = targetProfit
                    if allStats is None:
                        allStats = stats
                    else:
                        allStats = allStats.append(stats)
                    sys.stdout.write(s + "\t %.2f seconds" % (time.time() - start_time))
                    sys.stdout.flush()
            

allStats.to_csv('D:\\Dropbox\\My work\\krypl-project\\result\\customStrategy.tsv', index=False, sep='\t')

# Look on transactions

In [ ]:
len(strategy.exchange.transactions)

In [ ]:
winningTransactions = []
for i in range(int(len(strategy.exchange.transactions) / 2)):
    t1, t2 = strategy.exchange.transactions[i:i+2]
    if t1['amount'] < t2['amount']:
        winningTransactions += [t1, t2]

In [ ]:
len(winningTransactions)

In [ ]:
strategy.exchange.transactions[2:4]

In [ ]:
data_train.head()

In [ ]:
%matplotlib notebook
plot_transactions(ohlc_train, strategy.exchange.transactions[:])

for c in ['red', 'yellow', 'green', 'blue']:
    plt.plot(range(ohlc_train.shape[0]), ohlc_train[c], c=c)

In [ ]:
t = 70

transaction = strategy.exchange.transactions[t]
plotTransactionWithSurroundings(ohlc_train, strategy.historyLen, transaction)
transaction = strategy.exchange.transactions[t+1]
plotTransactionWithSurroundings(ohlc_train, strategy.historyLen, transaction)

In [ ]:
transaction

# Evaluate

In [ ]:
allStats = pd.read_csv('D:\\Dropbox\\My work\\krypl-project\\result\\customStrategy.tsv',sep='\t')

In [ ]:
allStats.query('totalProfit > 90 and winPercentage > 70').sort_values('numberOfTrades', ascending=False)

In [ ]:
allStats.sort_values('totalProfit', ascending=False).query('willingLoss < 0.1').query('winPercentage > 50')

In [ ]:
dataManager = deepcopy(data_test)
exchange = BackTestExchange(dataManager, deepcopy(wallet), 0.0025)
strategy = CustomStrategy(exchange, dataManager, 2016, contract_pair, 100, willingLoss=0.05,
                          distanceFromMaxThreshold1=0.06, distanceFromMaxThreshold2=0.03, targetProfit=0.01)
strategy.trade()
strategy.stats('usdt').report()